In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
from tqdm import tqdm

In [2]:
class GeradorDeNumerosAleatorios():
    def __init__(self, semente: int, a: int, c: int, m: int):
        self.semente = semente
        self.a = a
        self.c = c
        self.m = m

        self.ultimoGerado = semente

    def ObterFatoresPrimos(numero: int) -> set:
        """
        Calcula os fatores primos de um determinado número.
        Retorna um conjunto (set) com os fatores.
        """
        fatores = set()
        divisor = 2

        # Reduz o número por seus fatores de 2
        while numero % divisor == 0:
            fatores.add(divisor)
            numero //= divisor

        # Agora testa divisores ímpares
        divisor = 3
        while divisor * divisor <= numero:
            while numero % divisor == 0:
                fatores.add(divisor)
                numero //= divisor
            divisor += 2

        # Se o número restante for primo e maior que 2
        if numero > 2:
            fatores.add(numero)

        return fatores

    def EncontrarParesHullDobell(moduloM: int, numPares: int) -> list[tuple[int, int]]:
        """
        Procura exaustivamente por pares (a, c) que satisfazem o Teorema de Hull-Dobell
        para um dado módulo M, resultando em um Gerador Linear Congruencial de período máximo.

        Args:
            moduloM (int): O módulo 'M' do gerador.
            numPares (int): O número de pares (a, c) a serem encontrados.
                            Se for 0, procura por todos os pares possíveis.

        Returns:
            list[tuple[int, int]]: Uma lista de tuplas, onde cada tupla é um par (a, c) válido.
        """
        if moduloM <= 1:
            print("O módulo M deve ser maior que 1.")
            return []

        paresEncontrados = []

        # Pré-calcula os fatores primos de M para otimização
        fatoresPrimosDeM = ObterFatoresPrimos(moduloM)
        print(f"Módulo M = {moduloM}")
        print(f"Fatores primos de M: {fatoresPrimosDeM if fatoresPrimosDeM else 'Nenhum'}\n")

        # A barra de progresso irá iterar de 0 a M-1
        # O desc é a descrição que aparece ao lado da barra
        for a in tqdm(range(moduloM), desc=f"Procurando pares (a,c) para M={moduloM}"):
            # Condição 2 e 3 (dependem apenas de 'a')
            aMenosUm = a - 1

            # --- Verificação da Condição 3 ---
            # Se M é múltiplo de 4, a-1 também deve ser.
            if moduloM % 4 == 0 and aMenosUm % 4 != 0:
                continue # Falhou, vai para o próximo 'a'

            # --- Verificação da Condição 2 ---
            # a-1 deve ser divisível por todos os fatores primos de M
            condicao2Satisfeita = True
            for fator in fatoresPrimosDeM:
                if aMenosUm % fator != 0:
                    condicao2Satisfeita = False
                    break # Se falhar para um fator, não precisa testar os outros

            if not condicao2Satisfeita:
                continue # Falhou, vai para o próximo 'a'

            # Se 'a' passou nas condições, agora testamos 'c'
            for c in range(moduloM):
                # --- Verificação da Condição 1 ---
                # mdc(c, M) deve ser 1
                if math.gcd(c, moduloM) == 1:
                    # Se chegou aqui, o par (a, c) é válido!
                    paresEncontrados.append((a, c))

                    # Se já encontramos o número de pares desejado (e não é 0)
                    if numPares != 0 and len(paresEncontrados) >= numPares:
                        print(f"\nBusca concluída. {len(paresEncontrados)} pares encontrados.")
                        return paresEncontrados

        print(f"\nBusca concluída. {len(paresEncontrados)} pares encontrados.")
        return paresEncontrados

    def GerarAleatorios(self, n):
        ultimoGerado = self.semente
        numeros = pd.DataFrame(columns=["i", 'Número Gerado', 'Número Normalizado'])

        for i in range(0, n):
            ultimoGerado = (self.a * ultimoGerado + self.c) % self.m
            numeros = pd.concat([numeros, pd.DataFrame({'i': [i], 'Número Gerado': [ultimoGerado], 'Número Normalizado': [ultimoGerado / self.m]})], ignore_index=True)

        numeros.plot(x='i', y='Número Normalizado', kind='scatter')

        return numeros

    def GerarProximoAleatorio(self):
        self.ultimoGerado = (self.a * self.ultimoGerado + self.c) % self.m
        return self.ultimoGerado / self.m

class Simulador:
    def __init__(self, gerador: GeradorDeNumerosAleatorios, parada: int, numeroDeServidores: int, tamanhoDaFila: int, variacaoNoTempoDeChegada: tuple, variacaoNoTempoDeSaida: tuple, comecarEm: float):
        self.gerador = gerador
        self.parada = parada
        self.numeroDeServidores = numeroDeServidores
        self.tamanhoDaFila = tamanhoDaFila
        self.variacaoNoTempoDeChegada = variacaoNoTempoDeChegada
        self.variacaoNoTempoDeSaida = variacaoNoTempoDeSaida
        self.comecarEm = comecarEm

        self.eventos: list[Simulador.Evento] = []
        self.fila = self.Fila(tamanhoDaFila)
        self.perdas: int = 0
        self.tempoTotalDaExecucao: float = 0.0

        self.tempoDeExecucaoPorPosicoesOcupadas: list[float] = []
        for i in range(0, tamanhoDaFila + 1):
            self.tempoDeExecucaoPorPosicoesOcupadas.append(0.0)

        self.eventos.append(self.Evento("chegada", comecarEm))

    class Evento:
        def __init__(self, tipo: str, tempo: float):
            self.tipo = tipo
            self.tempo = tempo

    class Fila:
        def __init__(self, tamanho: int):
            self.tamanho = tamanho
            self.posicoesOcupadas = 0

    def Simular(self):
        numerosGerados = 0

        while numerosGerados < self.parada:
            proximoEvento = self.ProximoEvento()
            if proximoEvento == "chegada":
                self.ProcessaEventoDeChegada(proximoEvento)
            elif proximoEvento == "saida":
                self.ProcessaEventoDeSaida(proximoEvento)

        pass

    def ProcessaEventoDeChegada(self, evento):
        self.AcumulaTempoTotalDeExecucao(evento)
        if self.fila.posicoesOcupadas < self.fila.tamanho:
            self.fila.posicoesOcupadas += 1
            # calcular tempo? acho que sim, pois está alterando o número de posições ocupadas
            if self.fila.posicoesOcupadas <= self.numeroDeServidores:
                self.eventos.append(self.Evento("saida", self.tempoTotalDaExecucao + self.EstimaTempo("saida")))
                self.numeroDeServidores -= 1
                # calcular tempo? acho que não, pois a contagem é feito pelo número de pessoas na fila e não das sendo atendidas
        else:
            self.perdas += 1
            Escalonador

    def ProcessaEventoDeSaida(self):
        pass

    def ProximoEvento(self):
        if not self.eventos: return None
        return min(self.eventos, key=lambda e: e.tempo)

    def AcumulaTempoTotalDeExecucao(self, evento):
        self.tempoTotalDaExecucao += evento.tempo

    def EstimaTempo(self, tipo: str):
        if tipo == "chegada":
            inicio = self.variacaoNoTempoDeChegada[0]
            fim    = self.variacaoNoTempoDeChegada[1]
            return (fim - inicio) * self.gerador.GerarProximoAleatorio() + inicio
        elif tipo == "saida":
            inicio = self.variacaoNoTempoDeSaida[0]
            fim    = self.variacaoNoTempoDeSaida[1]
            return (fim - inicio) * self.gerador.GerarProximoAleatorio() + inicio



In [3]:
gerador = GeradorDeNumerosAleatorios(semente = 7, a = 1664525, c = 1013904223, m = 4294967296)
numeros = gerador.GerarAleatorios(n = 10)
numeros

Tempo total de execução: 500.0
Tempo por posições ocupadas: [496.7822276854422, 3.2177723145578057, 0.0, 0.0, 0.0, 0.0]
Probabilidades por posições ocupadas: [0.9935644553708843, 0.006435544629115612, 0.0, 0.0, 0.0, 0.0]
Perdas: 0
